In [1]:
import pandas as pd
import torch
import os
from models_utils.GLOBALS import files_directory
from RF_XGB.RandomForest import get_rf_data, train_random_forest
from models_utils.utils import convert_to_features

In [2]:
# get train data
data_type_1, data_type_2 = get_rf_data()

In [3]:
# train rf
rf_classifier_type1, label_encoder_type1 = train_random_forest(data_type_1, ['activity'], 200)
rf_classifier_type2, label_encoder_type2 = train_random_forest(data_type_2, ['activity'], 200)

Accuracy: 0.9660127106935618
Log Loss: 0.261437658904284
Accuracy: 0.9569854248133665
Log Loss: 0.3947428930042928


In [ ]:
# get and save results for test data
labels = ['brushing_teeth', 'idle', 'preparing_sandwich', 'reading_book', 'stairs_down', 'stairs_up', 'typing',
          'using_phone', 'using_remote_control', 'walking_freely', 'walking_holding_a_tray',
          'walking_with_handbag', 'walking_with_hands_in_pockets', 'walking_with_object_underarm',
          'washing_face_and_hands', 'washing_mug', 'washing_plate', 'writing']
results_list = []

for i, file_id in enumerate(pd.read_csv('sample_submission.csv')['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)
    if new_data.shape[1] == 4:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        data_x_tensor = torch.tensor(new_data["x"].values, dtype=torch.float32)
        data_y_tensor = torch.tensor(new_data["y"].values, dtype=torch.float32)
        data_z_tensor = torch.tensor(new_data["z"].values, dtype=torch.float32)
        res = rf_classifier_type1.predict_proba(convert_to_features(data_x_tensor, data_y_tensor, data_z_tensor))
        res_dict = dict(zip(label_encoder_type1.inverse_transform(rf_classifier_type1.classes_), res[0]))
    else:
        data_x_tensor = torch.tensor(new_data["x [m]"].values, dtype=torch.float32)
        data_y_tensor = torch.tensor(new_data["y [m]"].values, dtype=torch.float32)
        data_z_tensor = torch.tensor(new_data["z [m]"].values, dtype=torch.float32)
        res = rf_classifier_type2.predict_proba(convert_to_features(data_x_tensor, data_y_tensor, data_z_tensor))
        res_dict = dict(zip(label_encoder_type2.inverse_transform(rf_classifier_type2.classes_), res[0]))

    result_dict = {label: res_dict.get(label, 0) for label in labels}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)

results = pd.DataFrame(results_list, columns=['sample_id'] + labels)
results.to_csv('results_rf5.csv', index=False)